# ওপেন এআই মডেল ফাইন টিউনিং

এই নোটবুকটি ওপেন এআই-এর [ফাইন টিউনিং](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) ডকুমেন্টেশনে প্রদত্ত বর্তমান নির্দেশনার উপর ভিত্তি করে তৈরি।

ফাইন-টিউনিং আপনার অ্যাপ্লিকেশনের জন্য ফাউন্ডেশন মডেলগুলোর কর্মক্ষমতা উন্নত করে অতিরিক্ত ডেটা এবং নির্দিষ্ট ব্যবহার বা পরিস্থিতির সাথে সম্পর্কিত প্রসঙ্গের মাধ্যমে পুনরায় প্রশিক্ষণ দিয়ে। লক্ষ্য করুন যে প্রম্পট ইঞ্জিনিয়ারিং কৌশল যেমন _few shot learning_ এবং _retrieval augmented generation_ আপনাকে প্রাসঙ্গিক ডেটা দিয়ে ডিফল্ট প্রম্পট উন্নত করার সুযোগ দেয় গুণগত মান বাড়ানোর জন্য। তবে, এই পদ্ধতিগুলো লক্ষ্য ফাউন্ডেশন মডেলের সর্বোচ্চ টোকেন উইন্ডোর আকার দ্বারা সীমাবদ্ধ।

ফাইন-টিউনিংয়ের মাধ্যমে, আমরা কার্যত মডেলটিকে প্রয়োজনীয় ডেটা দিয়ে পুনরায় প্রশিক্ষণ দিচ্ছি (যাতে আমরা সর্বোচ্চ টোকেন উইন্ডোর মধ্যে ফিট হওয়ার চেয়ে অনেক বেশি উদাহরণ ব্যবহার করতে পারি) - এবং একটি _কাস্টম_ সংস্করণ মোতায়েন করছি যা আর ইনফারেন্স সময় উদাহরণ সরবরাহের প্রয়োজন হয় না। এটি কেবল আমাদের প্রম্পট ডিজাইনের কার্যকারিতা উন্নত করে (আমাদের টোকেন উইন্ডো অন্যান্য জিনিসের জন্য ব্যবহার করার আরও নমনীয়তা থাকে) বরং সম্ভাব্যভাবে আমাদের খরচও কমায় (ইনফারেন্স সময় মডেলে পাঠানোর জন্য প্রয়োজনীয় টোকেনের সংখ্যা কমিয়ে)।

ফাইন টিউনিংয়ের ৪টি ধাপ রয়েছে:
1. প্রশিক্ষণ ডেটা প্রস্তুত করুন এবং আপলোড করুন।
1. ফাইন-টিউনড মডেল পেতে প্রশিক্ষণ কাজ চালান।
1. ফাইন-টিউনড মডেল মূল্যায়ন করুন এবং গুণগত মানের জন্য পুনরাবৃত্তি করুন।
1. সন্তুষ্ট হলে ইনফারেন্সের জন্য ফাইন-টিউনড মডেল মোতায়েন করুন।

লক্ষ্য করুন যে সব ফাউন্ডেশন মডেল ফাইন-টিউনিং সমর্থন করে না - সর্বশেষ তথ্যের জন্য [OpenAI ডকুমেন্টেশন চেক করুন](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst)। আপনি পূর্বে ফাইন-টিউনড মডেলকেও পুনরায় ফাইন-টিউন করতে পারেন। এই টিউটোরিয়ালে, আমরা ফাইন-টিউনিংয়ের জন্য `gpt-35-turbo` কে আমাদের লক্ষ্য ফাউন্ডেশন মডেল হিসেবে ব্যবহার করব।

---


### ধাপ ১.১: আপনার ডেটাসেট প্রস্তুত করুন

চলুন এমন একটি চ্যাটবট তৈরি করি যা একটি উপাদানের সম্পর্কে প্রশ্নের উত্তর দিয়ে উপাদানের পর্যায় সারণী বুঝতে সাহায্য করে একটি লিমেরিকের মাধ্যমে। এই _সহজ_ টিউটোরিয়ালে, আমরা কেবল একটি ডেটাসেট তৈরি করব মডেলকে প্রশিক্ষণ দেওয়ার জন্য কিছু নমুনা প্রতিক্রিয়ার উদাহরণ দিয়ে যা ডেটার প্রত্যাশিত ফরম্যাট প্রদর্শন করে। বাস্তব জীবনের ব্যবহারে, আপনাকে অনেক বেশি উদাহরণের একটি ডেটাসেট তৈরি করতে হবে। আপনার অ্যাপ্লিকেশন ডোমেইনের জন্য যদি একটি ওপেন ডেটাসেট থাকে, তবে আপনি সেটি ব্যবহার করে ফাইন-টিউনিংয়ের জন্য পুনরায় ফরম্যাট করতে পারেন।

আমরা যেহেতু `gpt-35-turbo` এর উপর ফোকাস করছি এবং একক-টর্ন প্রতিক্রিয়া (চ্যাট সম্পূর্ণকরণ) খুঁজছি, তাই আমরা [এই প্রস্তাবিত ফরম্যাট](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) ব্যবহার করে উদাহরণ তৈরি করতে পারি যা OpenAI চ্যাট সম্পূর্ণকরণের প্রয়োজনীয়তাগুলি প্রতিফলিত করে। আপনি যদি বহু-টর্ন কথোপকথনের বিষয়বস্তু আশা করেন, তবে আপনি [বহু-টর্ন উদাহরণ ফরম্যাট](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst) ব্যবহার করবেন যা একটি `weight` প্যারামিটার অন্তর্ভুক্ত করে যা সংকেত দেয় কোন বার্তাগুলি ফাইন-টিউনিং প্রক্রিয়ায় ব্যবহার করা উচিত (বা নয়)।

আমরা এখানে আমাদের টিউটোরিয়ালের জন্য সহজ একক-টর্ন ফরম্যাট ব্যবহার করব। ডেটা [jsonl ফরম্যাটে](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) রয়েছে, প্রতি লাইনে ১টি রেকর্ড, প্রতিটি একটি JSON-ফরম্যাটেড অবজেক্ট হিসেবে উপস্থাপিত। নিচের অংশে ২টি রেকর্ড নমুনা হিসেবে দেখানো হয়েছে - সম্পূর্ণ নমুনা সেট (১০টি উদাহরণ) এর জন্য দেখুন [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) যা আমরা আমাদের ফাইন-টিউনিং টিউটোরিয়ালের জন্য ব্যবহার করব। **দ্রষ্টব্য:** প্রতিটি রেকর্ড _অবশ্যই_ এক লাইনে সংজ্ঞায়িত হতে হবে (সাধারণত ফরম্যাট করা JSON ফাইলে যেমন লাইন বিভক্ত করা হয় তা নয়)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

বাস্তব জীবনের ব্যবহারে ভাল ফলাফলের জন্য আপনাকে অনেক বড় উদাহরণ সেট প্রয়োজন হবে - এখানে প্রতিক্রিয়ার গুণমান এবং ফাইন-টিউনিংয়ের সময়/খরচের মধ্যে একটি সমঝোতা থাকবে। আমরা একটি ছোট সেট ব্যবহার করছি যাতে আমরা দ্রুত ফাইন-টিউনিং সম্পন্ন করতে পারি এবং প্রক্রিয়াটি ব্যাখ্যা করতে পারি। আরও জটিল ফাইন-টিউনিং টিউটোরিয়ালের জন্য দেখুন [এই OpenAI কুকবুক উদাহরণ](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst)।


---

### ধাপ ১.২ আপনার ডেটাসেট আপলোড করুন

ফাইলস API ব্যবহার করে ডেটা আপলোড করুন [যেমন এখানে বর্ণনা করা হয়েছে](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file)। লক্ষ্য করুন যে এই কোড চালানোর জন্য, আপনাকে প্রথমে নিম্নলিখিত ধাপগুলি সম্পন্ন করতে হবে:
 - `openai` পাইথন প্যাকেজ ইনস্টল করা (নিশ্চিত করুন আপনি সর্বশেষ ফিচারের জন্য >=0.28.0 সংস্করণ ব্যবহার করছেন)
 - `OPENAI_API_KEY` পরিবেশ ভেরিয়েবল আপনার OpenAI API কী সেট করা
আরও জানতে, কোর্সের জন্য প্রদত্ত [সেটআপ গাইড](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) দেখুন।

এখন, আপনার লোকাল JSONL ফাইল থেকে আপলোডের জন্য একটি ফাইল তৈরি করতে কোডটি চালান।


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


---

### ধাপ ২.১: SDK দিয়ে ফাইন-টিউনিং কাজ তৈরি করুন


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


---

### ধাপ ২.২: কাজের অবস্থা পরীক্ষা করুন

`client.fine_tuning.jobs` API দিয়ে আপনি কয়েকটি কাজ করতে পারেন:
- `client.fine_tuning.jobs.list(limit=<n>)` - শেষ nটি ফাইন-টিউনিং কাজের তালিকা দেখুন
- `client.fine_tuning.jobs.retrieve(<job_id>)` - একটি নির্দিষ্ট ফাইন-টিউনিং কাজের বিস্তারিত তথ্য পান
- `client.fine_tuning.jobs.cancel(<job_id>)` - একটি ফাইন-টিউনিং কাজ বাতিল করুন
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - কাজ থেকে সর্বোচ্চ nটি ইভেন্টের তালিকা দেখুন
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

প্রক্রিয়াটির প্রথম ধাপ হল _ট্রেনিং ফাইল যাচাই করা_ যাতে ডেটা সঠিক ফরম্যাটে আছে কিনা নিশ্চিত হওয়া যায়।


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


---

### ধাপ ২.৩: অগ্রগতি পর্যবেক্ষণের জন্য ইভেন্ট ট্র্যাক করুন  


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### ধাপ ২.৪: OpenAI ড্যাশবোর্ডে অবস্থা দেখুন


আপনি OpenAI ওয়েবসাইটে গিয়ে এবং প্ল্যাটফর্মের _Fine-tuning_ বিভাগ অন্বেষণ করে স্ট্যাটাসও দেখতে পারেন। এটি আপনাকে বর্তমান কাজের স্ট্যাটাস দেখাবে, এবং পূর্ববর্তী কাজের কার্যকরী রানগুলির ইতিহাস ট্র্যাক করতে দেবে। এই স্ক্রিনশটে, আপনি দেখতে পাচ্ছেন যে পূর্ববর্তী কার্যকরী রানটি ব্যর্থ হয়েছে, এবং দ্বিতীয় রানটি সফল হয়েছে। প্রেক্ষাপটের জন্য, এটি ঘটেছিল যখন প্রথম রানটি ভুল ফরম্যাট করা রেকর্ড সহ একটি JSON ফাইল ব্যবহার করেছিল - একবার ঠিক করার পর, দ্বিতীয় রানটি সফলভাবে সম্পন্ন হয় এবং মডেলটি ব্যবহারের জন্য উপলব্ধ করে তোলে।

![Fine-tuning job status](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba7e3f73a201f8712fae3cea1c08f7c7f12ca469c06d234122.bn.png)


আপনি ভিজ্যুয়াল ড্যাশবোর্ডে আরও নিচে স্ক্রোল করে স্ট্যাটাস বার্তা এবং মেট্রিক্সও দেখতে পারেন যেমনটি দেখানো হয়েছে:

| Messages | Metrics |
|:---|:---|
| ![Messages](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.bn.png) |  ![Metrics](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.bn.png)|


---

### ধাপ ৩.১: আইডি পুনরুদ্ধার করুন এবং কোডে ফাইন-টিউন করা মডেল পরীক্ষা করুন  


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


---

### ধাপ ৩.২: প্লেগ্রাউন্ডে ফাইন-টিউন করা মডেল লোড ও পরীক্ষা করুন

এখন আপনি দুটি উপায়ে ফাইন-টিউন করা মডেল পরীক্ষা করতে পারেন। প্রথমত, আপনি প্লেগ্রাউন্ডে যেতে পারেন এবং মডেলস ড্রপ-ডাউন থেকে আপনার নতুন ফাইন-টিউন করা মডেলটি তালিকা থেকে নির্বাচন করতে পারেন। অন্য বিকল্পটি হল ফাইন-টিউনিং প্যানেলে প্রদর্শিত "প্লেগ্রাউন্ড" অপশনটি ব্যবহার করা (উপরের স্ক্রিনশট দেখুন) যা নিম্নলিখিত _তুলনামূলক_ ভিউ চালু করে যা দ্রুত মূল্যায়নের জন্য ফাউন্ডেশন এবং ফাইন-টিউন করা মডেল সংস্করণগুলি পাশাপাশি দেখায়।

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-compare.56e06f0ad8922016497d39ced3d84ea296eec89073503f2bf346ec9718f913b5.bn.png)

আপনার প্রশিক্ষণ ডেটায় ব্যবহৃত সিস্টেম প্রসঙ্গটি সহজেই পূরণ করুন এবং আপনার পরীক্ষার প্রশ্ন দিন। আপনি লক্ষ্য করবেন যে উভয় পাশে একই প্রসঙ্গ এবং প্রশ্ন আপডেট হয়েছে। তুলনা চালান এবং আপনি তাদের আউটপুটের মধ্যে পার্থক্য দেখতে পাবেন। _দ্রষ্টব্য: ফাইন-টিউন করা মডেলটি আপনার উদাহরণে প্রদত্ত ফরম্যাটে প্রতিক্রিয়া প্রদান করে, যেখানে ফাউন্ডেশন মডেলটি কেবল সিস্টেম প্রম্পট অনুসরণ করে।_

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-launch.5a26495c983c6350c227e05700a47a89002d132949a56fa4ff37f266ebe997b2.bn.png)

আপনি লক্ষ্য করবেন যে তুলনাটি প্রতিটি মডেলের জন্য টোকেন গণনা এবং ইনফারেন্সের জন্য নেওয়া সময়ও প্রদান করে। **এই নির্দিষ্ট উদাহরণটি একটি সরলীকৃত যা প্রক্রিয়াটি দেখানোর জন্য তৈরি, কিন্তু বাস্তব বিশ্বের ডেটাসেট বা পরিস্থিতি প্রতিফলিত করে না**। আপনি দেখতে পাবেন যে উভয় নমুনায় একই সংখ্যক টোকেন রয়েছে (সিস্টেম প্রসঙ্গ এবং ব্যবহারকারীর প্রম্পট একই) এবং ফাইন-টিউন করা মডেল ইনফারেন্সের জন্য বেশি সময় নিচ্ছে (কাস্টম মডেল)।

বাস্তব বিশ্বের পরিস্থিতিতে, আপনি এই ধরনের খেলনা উদাহরণ ব্যবহার করবেন না, বরং বাস্তব ডেটার বিরুদ্ধে ফাইন-টিউন করবেন (যেমন, গ্রাহক সেবার জন্য পণ্য ক্যাটালগ) যেখানে প্রতিক্রিয়ার গুণমান অনেক বেশি স্পষ্ট হবে। _সেই_ প্রসঙ্গে, ফাউন্ডেশন মডেলের সাথে সমমানের প্রতিক্রিয়া গুণমান পেতে আরও কাস্টম প্রম্পট ইঞ্জিনিয়ারিং প্রয়োজন হবে যা টোকেন ব্যবহারে বৃদ্ধি এবং সম্ভাব্য ইনফারেন্সের প্রক্রিয়াকরণ সময় বাড়াবে। _এটি চেষ্টা করতে, OpenAI কুকবুকে ফাইন-টিউনিং উদাহরণগুলি দেখুন শুরু করার জন্য।_

---


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**অস্বীকৃতি**:  
এই নথিটি AI অনুবাদ সেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনূদিত হয়েছে। আমরা যথাসাধ্য সঠিকতার চেষ্টা করি, তবে স্বয়ংক্রিয় অনুবাদে ত্রুটি বা অসঙ্গতি থাকতে পারে। মূল নথিটি তার নিজস্ব ভাষায়ই কর্তৃত্বপূর্ণ উৎস হিসেবে বিবেচিত হওয়া উচিত। গুরুত্বপূর্ণ তথ্যের জন্য পেশাদার মানব অনুবাদ গ্রহণ করার পরামর্শ দেওয়া হয়। এই অনুবাদের ব্যবহারে সৃষ্ট কোনো ভুল বোঝাবুঝি বা ভুল ব্যাখ্যার জন্য আমরা দায়ী নই।
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
